In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import platform
from matplotlib.ticker import FuncFormatter
import plotly.graph_objects as go
from matplotlib import rcParams
import csv
import matplotlib.ticker as ticker
from plotly.subplots import make_subplots
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
import plotly.express as px

plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows 기본 한글 폰트
plt.rcParams['axes.unicode_minus'] = False     # 마이너스 기호 깨짐 방지


total22_df = pd.read_csv("전연령22_utf8.csv", low_memory=False)
senior22_df = pd.read_csv("노인승하차22_utf8.csv", low_memory=False)
total24_df = pd.read_csv("전연령24_utf8.csv", low_memory=False)
senior24_df = pd.read_csv("노인승하차24_utf8.csv", low_memory=False)
total23_df = pd.read_csv("전연령23_utf8.csv", low_memory=False)
senior23_df = pd.read_csv("노인승하차23_utf8.csv", low_memory=False)
#senior_care_df = pd.read_csv("senior_care_utf8.csv", low_memory=False) # 파일이 사라짐
sme_df = pd.read_csv("sme.csv", low_memory=False)
park_df = pd.read_csv("TB_PTP_PRK_M.csv", low_memory=False)
safety_df = pd.read_csv("안전사고_utf.csv", low_memory=False)
elderly_2020_2022 = pd.read_csv(
    "elderly2020_2022.csv",
    low_memory=False,
    quoting=csv.QUOTE_NONE,
    encoding="utf-8",
    on_bad_lines='skip'  # 문제 있는 라인은 건너뜀
)
elderly_2023_2025 = pd.read_csv(
    "elderly2023_2025.csv",
    low_memory=False,
    quoting=csv.QUOTE_NONE,
    encoding="utf-8",
    on_bad_lines='skip'
)
weather_raw = pd.read_csv("weath20_25_utf8.csv", encoding='utf-8')
rain_raw = pd.read_csv("rain20_25_utf8.csv", encoding='utf-8') # 강수량 파일명 확인 필요
import pandas as pd

# ----------------------------------------------------------
# [1] 주요 노인 이용 역 15곳 선정 및 데이터 입력
# ----------------------------------------------------------
# 선정 기준: 승하차 인원 상위권 + 사고 다발 우려 지역
data_expanded = {
    '역명': [
        '제기동', '청량리', '종로3가', '종로5가', '동묘앞',   # 1. 도심/시장 라인 (전통적 핫플레이스)
        '연신내', '수유', '미아사거리', '쌍문', '창동',     # 2. 강북 주거 벨트 (출발지 수요 많음)
        '신도림', '구로디지털단지', '신림', '사당',         # 3. 서남권 거점 (환승/주거)
        '고속터미널'                                      # 4. 대형 환승 허브 (복잡도 높음)
    ],
    '호선': [
        '1호선', '1호선', '1호선', '1호선', '1호선',
        '3호선', '4호선', '4호선', '4호선', '4호선',
        '2호선', '2호선', '2호선', '2호선',
        '3호선'
    ],
    # 엘리베이터 (E/V): 노인 이동의 필수 수단 (적을수록 위험)
    'EV_개수': [
        2, 4, 3, 2, 2,    # 도심권은 구조상 설치가 어려워 적은 편
        4, 3, 3, 2, 3,    # 주거지도 이용객 대비 부족
        4, 3, 3, 5,       # 유동인구가 워낙 많은 곳들
        3
    ],
    # 에스컬레이터 (E/S): 사고가 가장 많이 나는 시설
    'ES_개수': [
        6, 8, 12, 4, 6,
        16, 8, 8, 6, 8,
        14, 10, 8, 18,
        24                # 고터 등 환승역은 E/S가 매우 많음 (그만큼 사고 위험도 높음)
    ],
    # 휠체어리프트 (W/L): 엘리베이터 없는 곳의 대체 수단 (위험함)
    'WL_개수': [
        1, 2, 3, 1, 2,
        0, 1, 1, 2, 1,
        2, 1, 1, 2,
        1
    ]
}

# ----------------------------------------------------------
# [2] 데이터프레임 생성 및 저장
# ----------------------------------------------------------
df_facility_exp = pd.DataFrame(data_expanded)

# CSV 파일로 저장 (한글 깨짐 방지)
file_name = "station_facilities_expanded.csv"
df_facility_exp.to_csv(file_name, index=False, encoding='utf-8-sig')

print(f"✅ 확장된 편의시설 데이터 파일 '{file_name}' 생성 완료!")
print(f"   - 분석 대상 역: 총 {len(df_facility_exp)}개")
print("-" * 50)
print(df_facility_exp)


✅ 확장된 편의시설 데이터 파일 'station_facilities_expanded.csv' 생성 완료!
   - 분석 대상 역: 총 15개
--------------------------------------------------
         역명   호선  EV_개수  ES_개수  WL_개수
0       제기동  1호선      2      6      1
1       청량리  1호선      4      8      2
2      종로3가  1호선      3     12      3
3      종로5가  1호선      2      4      1
4       동묘앞  1호선      2      6      2
5       연신내  3호선      4     16      0
6        수유  4호선      3      8      1
7     미아사거리  4호선      3      8      1
8        쌍문  4호선      2      6      2
9        창동  4호선      3      8      1
10      신도림  2호선      4     14      2
11  구로디지털단지  2호선      3     10      1
12       신림  2호선      3      8      1
13       사당  2호선      5     18      2
14    고속터미널  3호선      3     24      1


In [48]:

# =========================================================
# [1] 데이터 초기화 (가장 중요!!)
# =========================================================
# ★★★ 여기에 엑셀/CSV 파일을 읽는 코드를 꼭 다시 넣거나, 
# 위쪽 셀에서 read_csv 부분을 다시 실행해주세요! ★★★

# (예시) safety_df = pd.read_csv("경로/파일이름.csv") 
# 만약 이미 로드하는 코드가 있다면 그 셀을 먼저 실행하고 오세요.

# 원본 훼손 방지를 위해 복사본 사용
df = safety_df.copy()

# =========================================================
# [2] 강력한 데이터 전처리 (문자열 자르기)
# =========================================================
# 1. '13:40:00' -> 문자열로 변환 -> ':' 기준으로 자르기 -> 앞부분('13') 선택
df['발생시간_숫자'] = df['발생시간'].astype(str).str.split(':').str[0]

# 2. 숫자로 변환 (에러나면 NaN) -> 0으로 채우기 -> 정수로 변환
# (숫자가 아닌 이상한 값이 있어도 강제로 처리합니다)
df['발생시간_숫자'] = pd.to_numeric(df['발생시간_숫자'], errors='coerce').fillna(0).astype(int)

print("🔍 데이터 복구 확인 (이제 숫자가 보여야 합니다):")
print(df[['발생시간', '발생시간_숫자']].head())
print("-" * 50)

# =========================================================
# [3] 시간대별 분석 (10시 ~ 17시)
# =========================================================
# 분석 대상 시간
target_hours = [11, 12, 13, 14, 15, 16, 17] # 8시간
duration_elderly = len(target_hours)
duration_others = 24 - duration_elderly

target_key = '노인집중(10-17시)'
other_key = '그외시간'

# 그룹핑
df['시간대구분'] = df['발생시간_숫자'].apply(
    lambda x: target_key if x in target_hours else other_key
)

# 집계 (사고유형 x 시간대)
type_counts = df.groupby(['사고유형', '시간대구분']).size().unstack(fill_value=0)

# 컬럼 방어 코드
if target_key not in type_counts.columns: type_counts[target_key] = 0
if other_key not in type_counts.columns: type_counts[other_key] = 0

# 시간당 발생 건수 계산
type_counts['노인집중_시간당'] = type_counts[target_key] / duration_elderly
type_counts['그외시간_시간당'] = type_counts[other_key] / duration_others

# 정렬
result_df = type_counts.sort_values(by='노인집중_시간당', ascending=False).head(10)

print("📊 [분석 결과]")
print(result_df[['노인집중_시간당', '그외시간_시간당']].round(2))

# =========================================================
# [4] 시각화
# =========================================================
fig = go.Figure()

fig.add_trace(go.Bar(
    x=result_df.index,
    y=result_df['노인집중_시간당'],
    name='노인 활동 시간 (10~17시)',
    marker_color='#E63946',
    text=result_df['노인집중_시간당'].map('{:.1f}건/h'.format),
    textposition='auto'
))

fig.add_trace(go.Bar(
    x=result_df.index,
    y=result_df['그외시간_시간당'],
    name='그 외 시간',
    marker_color='#1D3557',
    text=result_df['그외시간_시간당'].map('{:.1f}건/h'.format),
    textposition='auto'
))

fig.update_layout(
    title=dict(text="<b>시간당 안전사고 발생 밀도</b>", x=0.5),
    xaxis_title="사고 유형",
    yaxis_title="시간당 발생 건수 (건/Hour)",
    barmode='group',
    template="plotly_white"
)

fig.show()

🔍 데이터 복구 확인 (이제 숫자가 보여야 합니다):
       발생시간  발생시간_숫자
0  13:40:00       13
1  11:26:00       11
2  06:26:00        6
3  15:29:00       15
4  18:46:00       18
--------------------------------------------------
📊 [분석 결과]
시간대구분   노인집중_시간당  그외시간_시간당
사고유형                      
출입문관련      56.57     25.71
역구내 사고     32.57     17.06
열차내 사고     23.71     18.24
발빠짐        16.71     13.06
승강설비관련     14.29      5.47
기타          1.86      0.94


In [33]:


# ---------------------------------------------------------
# [1] 데이터 전처리 (오류 해결: '미상' 데이터 처리)
# ---------------------------------------------------------
df = safety_df.copy()

# 1. 시간 숫자 추출 ('13:40:00' -> 13)
#    - '미상' 같은 문자가 있으면 NaN(결측치)으로 변환됩니다.
df['시_임시'] = pd.to_numeric(
    df['발생시간'].astype(str).str.split(':').str[0], 
    errors='coerce'
).fillna(-1).astype(int) # NaN은 -1로 채우고 정수로 변환

# 2. 포맷 매핑 함수 (승하차 데이터 포맷과 통일)
def map_to_subway_time(h):
    if h == -1: return '시간미상' # '미상' 데이터 처리
    if h < 6: return '06시간대이전'
    if h >= 24: return '24시간대이후'
    return f"{h:02d}-{h+1:02d}시간대"

# 3. 적용
df['시간대구분'] = df['시_임시'].apply(map_to_subway_time)

# ---------------------------------------------------------
# [2] 데이터 집계
# ---------------------------------------------------------
# X축 순서 지정 (시간 순서대로 정렬)
time_order = [
    '06시간대이전', '06-07시간대', '07-08시간대', '08-09시간대', '09-10시간대',
    '10-11시간대', '11-12시간대', '12-13시간대', '13-14시간대', '14-15시간대',
    '15-16시간대', '16-17시간대', '17-18시간대', '18-19시간대', '19-20시간대',
    '20-21시간대', '21-22시간대', '22-23시간대', '23-24시간대', '24시간대이후',
    '시간미상'
]

# 그룹핑 (시간대구분 x 사고유형)
df_counts = df.groupby(['시간대구분', '사고유형']).size().reset_index(name='건수')


custom_colors = [
    '#4A90E2', # 선명한 블루 (출입문) - 가장 많음, 명확함
    '#F5A623', # 따뜻한 오렌지 (발빠짐) - 주의, 눈에 띔
    '#7ED321', # 싱그러운 그린 (승강설비) - 안정감, 보색 대비
    '#BD10E0', # 차분한 퍼플 (열차내) - 고급스러움
    '#D0021B', # 강조 레드 (역구내) - 경고, 포인트
    '#9B9B9B', # 중립 그레이 (기타) - 배경 역할
    '#50E3C2', # 민트 (추가 대비용)
    '#B8E986'  # 라임 (추가 대비용)
]

# 사고 유형별 총 건수를 구해 정렬 (많은 순서대로 쌓기 위함)
sorted_types = df_counts.groupby('사고유형')['건수'].sum().sort_values(ascending=False).index.tolist()

# ---------------------------------------------------------
# [3] 시각화 생성
# ---------------------------------------------------------
fig = go.Figure()

for i, accident_type in enumerate(sorted_types):
    subset = df_counts[df_counts['사고유형'] == accident_type]
    
    # 색상 순환 적용
    color = custom_colors[i % len(custom_colors)]
    
    # [디테일] 건수가 너무 적으면(2건 미만) 텍스트 숨기기 (깔끔함 유지)
    text_labels = [str(c) if c >= 2 else '' for c in subset['건수']]

    fig.add_trace(go.Bar(
        x=subset['시간대구분'],
        y=subset['건수'],
        name=accident_type,
        marker_color=color,
        
        # 텍스트 설정
        text=text_labels,
        textposition='inside', # 막대 안에 넣기
        textfont=dict(
            family="Malgun Gothic",
            size=10,
            color='white' # 파란 배경이므로 흰색 글씨로 통일 (깔끔함)
        ),
        
        # 마우스 오버 툴팁 (상세 정보는 여기서 확인)
        hovertemplate='<b>%{x}</b><br>' +
                      '유형: %{data.name}<br>' +
                      '건수: %{y}건<extra></extra>'
    ))

# ---------------------------------------------------------
# [4] 레이아웃 설정 (깔끔하게 정리)
# ---------------------------------------------------------
fig.update_layout(
    title=dict(
        text="<b>시간대별 안전사고 유형 분포</b>",
        x=0.5, y=0.95,
        font=dict(size=22, family="Malgun Gothic", color='#333')
    ),
    xaxis=dict(
        title="시간대",
        categoryorder='array',
        categoryarray=time_order,
        tickangle=-45,
        tickfont=dict(size=11),
        linecolor='#ddd' # 축 라인 연하게
    ),
    yaxis=dict(
        title="발생 건수 (건)",
        gridcolor='#f0f0f0', # 격자 아주 연하게
        zeroline=False
    ),
    # 범례 우측 배치
    legend=dict(
        title=dict(text="사고 유형", font=dict(size=20)), # 범례 제목 크기
        font=dict(size=16),       # ★ 범례 항목 글자 크기 (여기 숫자를 조절하세요)
        orientation="v",
        yanchor="top", y=1,
        xanchor="left", x=1.0,    # ★ 좌우 위치 (1.02 -> 1.0으로 수정, 더 줄이면 0.9...)
        bgcolor='rgba(255,255,255,0.5)'
    ),
    barmode='stack', # 누적 막대
    template="plotly_white", # 흰색 배경
    margin=dict(r=200, t=80), # 여백 조정
    height=600
)

fig.show()

In [47]:

# ---------------------------------------------------------
# [1] 데이터 전처리 (기존과 동일)
# ---------------------------------------------------------
df = safety_df.copy()

# 1. 시간 숫자 추출
df['시'] = df['발생시간'].astype(str).str.split(':').str[0]
df['시'] = pd.to_numeric(df['시'], errors='coerce').fillna(-1).astype(int)

# 2. 시간대 구분
core_hours = [11, 12, 13, 14, 15, 16, 17]
duration_core = 7
duration_other = 17

def classify_time_simple(h):
    if h in core_hours: return '노인 집중(11~17시)'
    else: return '그 외 시간'

df['시간대구분'] = df['시'].apply(classify_time_simple)
df['역명_검색용'] = df['발생역'].astype(str).str.split('(').str[0].str.replace('역$', '', regex=True).str.strip()

# ---------------------------------------------------------
# [2] 시각화 함수 (디자인 최적화 버전)
# ---------------------------------------------------------
def plot_density_comparison_clean(station_list, title_text):
    # 데이터 필터링 및 집계
    target_data = df[df['역명_검색용'].isin(station_list)].copy()
    counts = target_data.groupby(['역명_검색용', '시간대구분']).size().unstack(fill_value=0)
    
    # 컬럼 안전장치
    if '노인 집중(11~17시)' not in counts.columns: counts['노인 집중(11~17시)'] = 0
    if '그 외 시간' not in counts.columns: counts['그 외 시간'] = 0

    # 밀도 계산 (핵심)
    counts['밀도_노인'] = counts['노인 집중(11~17시)'] / duration_core
    counts['밀도_그외'] = counts['그 외 시간'] / duration_other
    
    # 순서 정렬
    valid_stations = [st for st in station_list if st in counts.index]
    counts = counts.loc[valid_stations]

    # --- 그래프 그리기 ---
    fig = go.Figure()

    # 막대 1: 노인 집중 (빨강)
    fig.add_trace(go.Bar(
        x=counts.index,
        y=counts['밀도_노인'],
        name='노인 집중 (7시간)',
        marker_color='#EF5350', 
        text=counts['밀도_노인'].map('{:.2f}'.format),
        textposition='inside',      
        insidetextanchor='middle',  
        textfont=dict(color='white', size=13, family="Arial Black"),
        hovertemplate='<b>%{x}</b><br>노인집중: %{y:.2f}건/h'
    ))

    # 막대 2: 그 외 시간 (남색)
    fig.add_trace(go.Bar(
        x=counts.index,
        y=counts['밀도_그외'],
        name='그 외 시간 (17시간)',
        marker_color='#2C3E50',
        text=counts['밀도_그외'].map('{:.2f}'.format),
        textposition='inside',      
        insidetextanchor='middle',  
        textfont=dict(color='white', size=13, family="Arial Black"),
        hovertemplate='<b>%{x}</b><br>그외시간: %{y:.2f}건/h'
    ))

    fig.update_layout(
        title=dict(
            text=f"<b>{title_text}</b>", # 제목
            x=0.5, y=0.95,
            font=dict(size=22, family="Malgun Gothic", color='#333')
        ),
        xaxis=dict(
            title="역명",
            tickfont=dict(size=14, family="Malgun Gothic", weight='bold'),
            linecolor='#ddd'
        ),
        yaxis=dict(
            title="시간당 발생 건수 (건/Hour)",
            gridcolor='#f0f0f0',
            zeroline=False
        ),
        legend=dict(
            orientation="h",       
            yanchor="bottom", y=1.02, 
            xanchor="center", x=0.5,  
            bgcolor='rgba(255,255,255,0)'
        ),
        barmode='group',
        template="plotly_white",
        margin=dict(t=100, b=50, l=50, r=50),
        height=500
    )
    
    fig.add_annotation(
        text="※ 막대 높이 = (총 사고 건수) ÷ (시간대 길이)",
        xref="paper", yref="paper",
        x=1.0, y=-0.25, 
        showarrow=False,
        font=dict(size=11, color="gray"),
        align="right"
    )

    fig.show()

# ---------------------------------------------------------
# [3] 실행 (총 3개 그래프 출력)
# ---------------------------------------------------------

# ① 승차 후보 1 (09시 기준 Top 5)
boarding_09 = ['연신내', '쌍문', '창동', '까치산', '수유']
print("📊 1. 승차 후보군 A (09-10시 기준)")
plot_density_comparison_clean(boarding_09, "[승차후보 A] 09시 기준 Top 5 : 사고 밀도 비교")

# ② 승차 후보 2 (10시 기준 Top 5)
# (※ 만약 10시 기준 Top 5 역이 다르다면 아래 리스트를 수정하세요. 예: 미아사거리 등)
boarding_10 = ['연신내', '쌍문', '창동', '수유', '미아사거리'] 
print("📊 2. 승차 후보군 B (10-11시 기준)")
plot_density_comparison_clean(boarding_10, "[승차후보 B] 10시 기준 Top 5 : 사고 밀도 비교")

# ③ 하차 확정 (12시 기준 Top 5)
alighting_12 = ['종로3가', '제기동', '청량리', '종로5가', '회현']
print("📊 3. 하차 확정군 (12-13시 기준)")
plot_density_comparison_clean(alighting_12, "[하차확정] 12시 기준 Top 5 : 사고 밀도 비교")

📊 1. 승차 후보군 A (09-10시 기준)


📊 2. 승차 후보군 B (10-11시 기준)


📊 3. 하차 확정군 (12-13시 기준)


In [50]:

# ---------------------------------------------------------
# [1] 데이터 전처리
# ---------------------------------------------------------
df = safety_df.copy()

# 1. 시간 숫자 추출
df['시'] = df['발생시간'].astype(str).str.split(':').str[0]
df['시'] = pd.to_numeric(df['시'], errors='coerce').fillna(-1).astype(int)

# 2. 시간대 구분
core_hours = [11, 12, 13, 14, 15, 16, 17]
duration_core = 7
duration_other = 17

def classify_time_simple(h):
    if h in core_hours: return '노인(7h)'
    else: return '그외(17h)'

df['시간대구분'] = df['시'].apply(classify_time_simple)
df['역명_검색용'] = df['발생역'].astype(str).str.split('(').str[0].str.replace('역$', '', regex=True).str.strip()

# ---------------------------------------------------------
# [2] 시각화 함수 (수정됨: .copy() 추가)
# ---------------------------------------------------------
def plot_density_breakdown(station_list, title_text):
    # 1. 데이터 필터링
    target_data = df[df['역명_검색용'].isin(station_list)].copy()
    
    # 2. 집계
    counts = target_data.groupby(['역명_검색용', '시간대구분', '사고유형']).size().reset_index(name='건수')
    
    # 3. 밀도 계산
    def calculate_density(row):
        if row['시간대구분'] == '노인(7h)':
            return row['건수'] / duration_core
        else:
            return row['건수'] / duration_other
            
    counts['밀도'] = counts.apply(calculate_density, axis=1)
    
    # 4. 시각화 준비
    fig = go.Figure()
    
    accident_colors = {
        '출입문관련': '#4A90E2', 
        '발빠짐': '#F5A623',   
        '승강설비관련': '#7ED321', 
        '열차내 사고': '#BD10E0', 
        '역구내 사고': '#D0021B', 
        '기타': '#9B9B9B'       
    }
    
    sorted_types = counts.groupby('사고유형')['밀도'].sum().sort_values(ascending=False).index.tolist()
    
    # 5. 그래프 그리기
    for accident_type in sorted_types:
        # ★ [수정 포인트] 뒤에 .copy()를 붙여서 경고 원천 차단!
        subset = counts[counts['사고유형'] == accident_type].copy()
        
        # 역 순서 정렬
        subset['역명_검색용'] = pd.Categorical(subset['역명_검색용'], categories=station_list, ordered=True)
        subset = subset.sort_values(['역명_검색용', '시간대구분'], ascending=[True, False])
        
        fig.add_trace(go.Bar(
            x=[subset['역명_검색용'], subset['시간대구분']], 
            y=subset['밀도'],
            name=accident_type,
            marker_color=accident_colors.get(accident_type, '#9B9B9B'),
            text=subset['건수'],
            textposition='inside',
            textfont=dict(color='white', size=10),
            hovertemplate='<b>%{x}</b><br>유형: %{data.name}<br>밀도: %{y:.2f} (실제 %{text}건)<extra></extra>'
        ))

    fig.update_layout(
        title=dict(
            text=f"<b>{title_text} : 시간당 사고 밀도 및 원인 분석</b>",
            x=0.5, y=0.95,
            font=dict(size=20, family="Malgun Gothic", color='#333')
        ),
        xaxis=dict(
            title="역명 / 시간대",
            tickfont=dict(size=12),
            linecolor='#ddd'
        ),
        yaxis=dict(
            title="시간당 발생 건수 (건/Hour)",
            gridcolor='#f0f0f0',
            zeroline=False
        ),
        legend=dict(
            orientation="h",       
            yanchor="bottom", y=1.02, 
            xanchor="center", x=0.5,  
            title_text=""
        ),
        barmode='stack',
        template="plotly_white",
        margin=dict(t=100, b=80),
        height=550
    )
    
    fig.add_annotation(
        text="※ 막대 높이: 시간당 사고 밀도 / 막대 내부 숫자: 실제 발생 건수",
        xref="paper", yref="paper",
        x=1.0, y=-0.3, 
        showarrow=False,
        font=dict(size=11, color="gray"),
        align="right"
    )

    fig.show()

# ---------------------------------------------------------
# [3] 실행
# ---------------------------------------------------------

# ① 승차 후보 A (09시 기준)
boarding_09 = ['연신내', '쌍문', '창동', '까치산', '수유']
print("📊 1. 승차 후보군 A (09시 기준)")
plot_density_breakdown(boarding_09, "[승차 A] 09시 Top 5")

# ② 승차 후보 B (10시 기준)
boarding_10 = ['연신내', '쌍문', '창동', '수유', '미아사거리'] 
print("📊 2. 승차 후보군 B (10시 기준)")
plot_density_breakdown(boarding_10, "[승차 B] 10시 Top 5")

# ③ 하차 확정 (12시 기준)
alighting_12 = ['종로3가', '제기동', '청량리', '종로5가', '회현']
print("📊 3. 하차 확정군 (12시 기준)")
plot_density_breakdown(alighting_12, "[하차] 12시 Top 5")

📊 1. 승차 후보군 A (09시 기준)


📊 2. 승차 후보군 B (10시 기준)


📊 3. 하차 확정군 (12시 기준)
